## Scrapping

In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

from newspaper import Article
import re

In [ ]:
host = 'localhost'
port = '3306'
username = 'root'
password = ''
database = 'news'

# Create Connection to database
# engine = create_engine('mysql+pymysql://'+username+
# ':'+password+'@'+host+':'+port+'/'+database)

# Note: We use pymysql instead of sqlalchemy because sqlalchemy
# somehow don't allow the text query. Strange bug.
conn = pymysql.connect(
    host=host,
    port=int(port),
    user=username,
    passwd=password,
    db=database,
    charset='utf8mb4')
'''engine = create_engine('mysql+pymysql://root: @localhost:3306
/news')'''

engine = create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database)

def run(sql):
    df = pd.read_sql_query(sql, conn)
    return df

In [ ]:
data = run("SELECT * FROM crime_news")
data['SOURCEURL']

### Scrapping masukan data ke tabel scrapping

Colum table
gid (Varchar), author (text), title (text), text (text), keywords (text), summary (text)


In [ ]:
number = 0
for d in data['SOURCEURL']:
    print('Data:',number)
    number += 1
    gid = data['GLOBALEVENTID'][number-1] 
    try:
        article = Article(d)
        article.download()
        article.parse()
        article.nlp()
        
        authors = ''.join(article.authors)
        title = ''.join(article.title)
        text = ''.join(article.text)
        keywords = ', '.join(article.keywords)
        summary = ''.join(article.summary)
        
        
        insert_string = "INSERT INTO scraping VALUES ('"+str(gid)+"', '"+re.sub(r"[^a-zA-Z0-9,;.]+", ' ', authors)+"', '"+re.sub(r"[^a-zA-Z0-9,;.]+", ' ', title)+"', '"+re.sub(r"[^a-zA-Z0-9,;.]+", ' ', text)+"', '"+re.sub(r"[^a-zA-Z0-9,;.]+", ' ', keywords)+"', '"+re.sub(r"[^a-zA-Z0-9,;.]+", ' ', summary)+"')"
    
        engine.execute(insert_string)
    except:
        print("Error:", number)